In [1]:
!pip install -q transformers accelerate noisereduce
!pip install -q pydub datasets soundfile pandas torch torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour

In [2]:
%%capture captured_output
%%time
import io
import os
import time
import numpy as np
import pandas as pd
import soundfile as sf
import torch
import tqdm
import subprocess
import noisereduce as nr
from datasets import load_dataset, Audio
from transformers import pipeline


HF_MODEL_ID = "EYEDOL/SALAMA_NEWMEDT"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
AUDIO_SAVE_DIR = "waves"
os.makedirs(AUDIO_SAVE_DIR, exist_ok=True)

print(f"Using device: {device}")
print(f"Loading Whisper model from Hugging Face: {HF_MODEL_ID}")

2025-10-07 08:42:00.328669: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759826520.551729      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759826520.617470      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:


# LOAD MODEL PIPELINE

asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model=HF_MODEL_ID,
    torch_dtype=torch_dtype,
    device=device,
)
print("Whisper model pipeline loaded successfully.")

# LOAD DATASET
print("Loading dataset...")
ds = load_dataset('sartifyllc/Sartify_ITU_Zindi_Testdataset', split="test")
ds = ds.cast_column("audio", Audio(sampling_rate=16000, decode=False))
print("Dataset loaded.")

# NORMALIZATION FUNCTION
def normalize_audio(audio, target_lufs=-23.0):
    rms = np.sqrt(np.mean(audio**2))
    if rms > 0:
        target_rms = 10**(target_lufs/20)
        audio = audio * (target_rms / rms)
    return np.clip(audio, -1.0, 1.0)

# GPU MEMORY FUNCTION
def get_gpu_memory():
    """Return current GPU memory usage (MiB)."""
    try:
        result = subprocess.check_output(
            ["nvidia-smi", "--query-gpu=memory.used", "--format=csv,noheader,nounits"]
        )
        return int(result.decode("utf-8").strip().split("\n")[0])
    except Exception:
        return 0

# PREPARE AUDIO FILES
print("Preparing audio files...")
audio_paths = []

for item in tqdm.tqdm(ds):
    audio_bytes = item['audio']['bytes']
    record_id = item['record_id']
    out_path = f"{AUDIO_SAVE_DIR}/{record_id}.wav"

    audio_array, sr = sf.read(io.BytesIO(audio_bytes))
    audio_array = normalize_audio(audio_array)
    sf.write(out_path, audio_array, sr)

    audio_paths.append({
        "path": out_path,
        "filename": item['filename'],
        "length_sec": len(audio_array) / sr
    })

print(f"{len(audio_paths)} audio files prepared for transcription.")
print("Starting transcription and measurement...")

# MAIN INFERENCE LOOP (RTFx + GPU MEMORY)
results = []
rtf_values = []
mem_usages = []

for audio_item in tqdm.tqdm(audio_paths):
    path = audio_item['path']
    clip_length = audio_item['length_sec']

    torch.cuda.reset_peak_memory_stats()
    start_mem = get_gpu_memory()
    start_time = time.time()

    try:
        transcription = asr_pipeline(path)
    except Exception as e:
        transcription = {"text": f"ERROR: {e}"}

    end_time = time.time()
    end_mem = get_gpu_memory()
    peak_mem = torch.cuda.max_memory_allocated() / 1024**2  # MB

    decoding_time = end_time - start_time
    rtf = clip_length / decoding_time

    results.append({
        "filename": audio_item['filename'],
        "text": transcription['text'],
        "clip_length_sec": clip_length,
        "decoding_time_sec": decoding_time,
        "RTFx": rtf,
        "Peak_Memory_MiB": peak_mem
    })

    rtf_values.append(rtf)
    mem_usages.append(max(peak_mem, end_mem - start_mem))

    print(f"{audio_item['filename']} | RTFx: {rtf:.2f} | Peak GPU Mem: {peak_mem:.2f} MiB")

# AGGREGATE METRICS
mean_rtf = np.mean(rtf_values)
peak_gpu_mem = np.max(mem_usages)

print("\n====================== FINAL RESULTS ======================")
print(f"Mean Real-Time Factor (RTFx): {mean_rtf:.3f}")
print(f"Peak GPU Memory Usage: {peak_gpu_mem:.2f} MiB")
print("===========================================================")

# SAVE RESULTS
pd.DataFrame(results).to_csv("rtfx_results.csv", index=False)

with open("rtfx_report.txt", "w") as f:
    f.write(f"Model: {HF_MODEL_ID}\n")
    f.write(f"Mean RTFx: {mean_rtf:.3f}\n")
    f.write(f"Peak GPU Memory (MiB): {peak_gpu_mem:.2f}\n")
    f.write("Hardware: Single NVIDIA T4 GPU\n")

print("\n Results saved to rtfx_report.txt and rtfx_results.csv")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

da4f005f-aac4-4724-b7a7-cb0ba925f282.wav | RTFx: 3.33 | Peak GPU Mem: 1529.10 MiB


/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:604: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
100%|█████████▉| 4088/4089 [2:33:42<00:02,  2.71s/it]

3931cc72-c9bb-45ce-8953-17b00dc4d800.wav | RTFx: 2.83 | Peak GPU Mem: 1529.57 MiB


/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:604: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
100%|██████████| 4089/4089 [2:33:44<00:00,  2.26s/it]

989732dc-3589-4c21-816c-f8e032ec0821.wav | RTFx: 3.26 | Peak GPU Mem: 1528.86 MiB

====================== FINAL RESULTS ======================
Mean Real-Time Factor (RTFx): 2.961
Peak GPU Memory Usage: 1738.00 MiB

 Results saved to rtfx_report.txt and rtfx_results.csv
